In [ ]:
#default_exp core

In [ ]:
#export
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from skimage.feature import peak_local_max

In [ ]:
#export
def expand_dim_to_3(arr):
    if arr.ndim == 2:
        return np.expand_dims(arr, axis=-1)
    elif arr.ndim == 3:
        return arr
    else:
        raise ValueError()

In [ ]:
# export

def elliptic_region(img, center, axes=None, angle=0, r=None, as_image=False, crop=False):
    """Returns the pixels in a elliptic region of a given image.
    
    Parameters:
    - img: 
    - center:
    - axes:
    - angle:
    - r:
    - as_image
    - crop
    """
    assert axes is not None or r is not None, "Either axes or radius must be passed"
    
    cx, cy = center
    a, b = axes if r is None else r,r
    
    width = img.shape[0]
    height = img.shape[1]
    mask = np.zeros((width, height), np.uint8) 

    circle_img = cv.ellipse(mask, center, axes, angle, color=(255,255,255), thickness=-1, startAngle=0, endAngle=360)
    
    masked_data = cv.bitwise_and(img, img, mask=circle_img)
    if crop:
        m = max(a,b)//2 #margin
        masked_data = masked_data[cy-m:cy+m+1, cx-m:cx+m+1]
        
    masked_data = expand_dim_to_3(masked_data)
    if as_image:
        return masked_data
    else:
        pixels = masked_data[masked_data.sum(axis=2)!=0]
#         pixels = masked_data[masked_data!=(0,0,0)]
        return pixels 


In [ ]:
#export
def regions(img, center, axes_in, axes_out, angle):
    full_region = elliptic_region(img, center, axes_out, angle=angle, as_image=True)

    blank_img = np.ones(full_region.shape[:2], dtype='uint8') * 255
    mask = elliptic_region(blank_img, center=center, axes=axes_in, as_image=True, angle=angle)

    outer_region = cv.bitwise_and(full_region, full_region.copy(), mask=cv.bitwise_not(mask))
    outer_region = expand_dim_to_3(outer_region)
    outer_region = outer_region[outer_region.sum(axis=2)!=0]
    
    inner_region = elliptic_region(full_region, center=center, axes=axes_in, as_image=False, angle=angle)
    return inner_region, outer_region

In [ ]:
#export
def seperability_filter(img, axes_in, axes_out, angle):
    height, width = img.shape[:2]
    result = np.zeros((height, width))
    
    for x in tqdm(range(axes_out[0], width-axes_out[0])):
        for y in range(axes_out[1], height-axes_out[1]):
            
            center = x, y
            r1_pixels, r2_pixels = regions(img, center, axes_in, axes_out, angle)

            
#             r1_pixels = intensity_region_1(img, x, y, r_in, True)
#             r2_pixels = intensity_region_2(img, x, y, r_in, r_out, True)
            if r1_pixels.size==0 or r2_pixels.size==0:
                result[y, x] = 0
                continue
            
            r1_mean = r1_pixels.mean(axis=0, keepdims=True).T
            r2_mean = r2_pixels.mean(axis=0, keepdims=True).T
            
            n_r1 = len(r1_pixels)
            n_r2 = len(r2_pixels)
            
            n_overall = n_r1 + n_r2
            overall_mean = r1_mean * n_r1/n_overall + r2_mean * n_r2/n_overall
                        
            S_B = n_overall * (r1_mean - overall_mean).dot((r1_mean - overall_mean).T) + \
                n_overall * (r2_mean - overall_mean).dot((r2_mean - overall_mean).T)
            
            S_T = np.cov(np.concatenate([r1_pixels, r2_pixels]).T)
            if np.isnan(S_B).any() or np.isnan(S_T).any():
                result[y, x]  = 0
            elif S_B.size==1:
                result[y, x]  = S_B / S_T
            else:
                result[y, x]  = np.trace(S_B) / np.trace(S_T)
                
    return result